# Delete Rejected Exposures

How to remove original movies and motion-corrected micrographs that were culled in an Exposure Curation job.

```{danger}
Improper use of this code may result in data loss. Please ensure the relevant data is backed up.
```

First, initialize the `CryoSPARC` client.

In [1]:
from cryosparc.tools import CryoSPARC

cs = CryoSPARC(host="cryoem0.sbi", base_port=40000)
assert cs.test_connection()

Connection succeeded to CryoSPARC command_core at http://cryoem0.sbi:40002
Connection succeeded to CryoSPARC command_vis at http://cryoem0.sbi:40003
Connection succeeded to CryoSPARC command_rtp at http://cryoem0.sbi:40005


Select a project and find its directory.

In [2]:
from pathlib import Path

project = cs.find_project("P251")
project_dir = Path(project.dir())

Select a "Manually Curate Exposures" job within the selected project and load its `exposures_rejected` output.

In [3]:
job = project.find_job("J93")
exposures_rejected = job.load_output("exposures_rejected")

```{note}
To instead clean up rejected exposures from a CryoSPARC Live session, first export exposures from the Session view at Details > Actions > Export Exposures. This results in a "Live Exposure Export" job in the session workspace. Select that job and load the `rejected_exposures` output instead.
```

The following loop prints the absolute path of associated files that may be deleted for each exposure.

In [4]:
for exposure in exposures_rejected.rows():
    for group in (
        "movie_blob",
        "micrograph_blob",
        "micrograph_blob_non_dw",
        "background_blob",
        "micrograph_thumbnail_blob_1x",
        "micrograph_thumbnail_blob_2x",
    ):
        field = group + "/path"
        if field in exposure:
            rel_path = exposure[field]
            abs_path = project_dir / rel_path
            print(rel_path)
            # abs_path.resolve().unlink()  # DANGER: UNCOMMENT TO PERFORM CLEANUP

J1/imported/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames.tif
J2/motioncorrected/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames_patch_aligned_doseweighted.mrc
J2/motioncorrected/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames_patch_aligned.mrc
J2/motioncorrected/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames_background.mrc
J2/thumbnails/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames_thumb_@1x.png
J2/thumbnails/009270517818331954156_14sep05c_00024sq_00003hl_00002es.frames_thumb_@2x.png
J1/imported/018045652812341181121_14sep05c_00024sq_00003hl_00005es.frames.tif
J2/motioncorrected/018045652812341181121_14sep05c_00024sq_00003hl_00005es.frames_patch_aligned_doseweighted.mrc
J2/motioncorrected/018045652812341181121_14sep05c_00024sq_00003hl_00005es.frames_patch_aligned.mrc
J2/motioncorrected/018045652812341181121_14sep05c_00024sq_00003hl_00005es.frames_background.mrc
J2/thumbnails/018045652812341181121_14sep05c_00024

To delete the rejected files, either un-comment the very last line and re-run the loop, or manually `rm` each path from the command line. For the latter strategy, resolve symlinks to ensure the original data gets deleted:

```sh
cd /path/to/project
rm -f $(readlink -f ${RELATIVE_PATH_TO_DELETE})  # for each file to delete...
```